In [126]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.graph_objs as go
import geocoder
import requests
from time import sleep
from bs4 import BeautifulSoup
import requests
import qgrid

In [138]:
#Importing data
bus = "Business_Licenses_-_Current_Active_cleaned.csv"
owners = "Business_Owners.csv"
bus_df = pd.read_csv(bus, encoding="ISO-8859-1", dtype='unicode')
owners_df = pd.read_csv(owners, encoding="ISO-8859-1", dtype='unicode')

In [139]:
#Convert all Dates to Datetime:
bus_df['APPLICATION CREATED DATE'] = pd.to_datetime(bus_df['APPLICATION CREATED DATE'])
bus_df['APPLICATION REQUIREMENTS COMPLETE'] = pd.to_datetime(bus_df['APPLICATION REQUIREMENTS COMPLETE'])
bus_df['PAYMENT DATE'] = pd.to_datetime(bus_df['PAYMENT DATE'])
bus_df['LICENSE TERM EXPIRATION DATE'] = pd.to_datetime(bus_df['LICENSE TERM EXPIRATION DATE'])
bus_df['LICENSE APPROVED FOR ISSUANCE'] = pd.to_datetime(bus_df['LICENSE APPROVED FOR ISSUANCE'])
bus_df['DATE ISSUED'] = pd.to_datetime(bus_df['DATE ISSUED'])
bus_df['LICENSE TERM START DATE'] = pd.to_datetime(bus_df['LICENSE TERM START DATE'])

In [140]:
owners_df.head()

,Account Number,Legal Name,Owner First Name,Owner Middle Initial,Owner Last Name,Suffix,Legal Entity Owner,Title
0,6,JOHN SCHALLER,JOHN,J.,SCHALLER,NaN,NaN,SOLE PROPRIETOR
1,56082,HELLENIC FOUNDATION,JAMES,NaN,SOUKOULIS,NaN,NaN,TREASURER
2,43,"GREENVILLE MISSISSIPPI CLUB, INC.",ELECTA,M,IVY,NaN,NaN,PRESIDENT
3,43,"GREENVILLE MISSISSIPPI CLUB, INC.",ROSEMARY,NaN,GARDNER,NaN,NaN,TREASURER
4,43,"GREENVILLE MISSISSIPPI CLUB, INC.",TOMMIE,J,RYHAL,NaN,NaN,SECRETARY


In [142]:
#Selects Columns that Matter
owners_df = owners_df[["Account Number","Owner First Name","Owner Middle Initial","Owner Last Name","Title"]]

In [143]:
#Remove all those rows with no names.
owners_df = owners_df.dropna(how="all",subset=["Owner First Name"]).reset_index(drop=True)
owners_df = owners_df.dropna(how="all",subset=["Owner Last Name"]).reset_index(drop=True)

In [144]:
#Preparing the Data so we can combine First Name with Last Name
owners_df["Owner Middle Initial"] = owners_df["Owner Middle Initial"].replace(np.nan, '', regex=True)

In [145]:
#Combine the First & Last name
owners_df["Name"] = owners_df["Owner First Name"] + " " + owners_df["Owner Middle Initial"] + " " + owners_df["Owner Last Name"]

In [146]:
#Widget Feature to help filter pandas dataframe

#1. First install qgrid - " pip install qgrid "
import qgrid

qgrid_widget = qgrid.show_grid(owners_df, show_toolbar= True, grid_options={'forceFitColumns':False, 'maxVisibleRows': 15})
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [147]:
#Used Qgrid to filter "Title" column for Unique Owners ("PRESIDENT, CEO, MEMBERS, MANAGERS, etc.").
True_owners = qgrid_widget.get_changed_df().reset_index(drop=True)

In [148]:
#Used Qgrid to filter "Title" column for everyone (excludes NaN,other,N/A).
#NOTE***: Go back and refilter qgrid_widget and then run this cell
everyone = qgrid_widget.get_changed_df().reset_index(drop=True)

In [149]:
#Adjusting the merging column
True_owners = True_owners.rename(columns={"Account Number":"ACCOUNT NUMBER"})
everyone = everyone.rename(columns={"Account Number":"ACCOUNT NUMBER"})

In [150]:
#Finally merging our DatFrames with the Business Licnese's Dataframe.
own_bus = bus_df.merge(True_owners,on="ACCOUNT NUMBER",how="left")
everyone_bus = bus_df.merge(everyone,on="ACCOUNT NUMBER",how="left")

In [157]:
#Capitalizing everything that matters (DBA & NAME).

own_bus["DOING BUSINESS AS NAME"] = own_bus["DOING BUSINESS AS NAME"].str.upper()
everyone_bus["DOING BUSINESS AS NAME"] = everyone_bus["DOING BUSINESS AS NAME"].str.upper()

own_bus["Name"] = own_bus["Name"].str.upper()
everyone_bus["Name"] = everyone_bus["Name"].str.upper()

own_bus["Owner First Name"] = own_bus["Owner First Name"].str.upper()
everyone_bus["Owner First Name"] = everyone_bus["Owner First Name"].str.upper()

own_bus["Owner Last Name"] = own_bus["Owner Last Name"].str.upper()
everyone_bus["Owner Last Name"] = everyone_bus["Owner Last Name"].str.upper()

In [159]:
#EXPORT DATA: CSV & XLSX FOR Tableau
own_bus.to_csv("OWNERS_BUSINESS_ONLY.csv",header=True, index = False)
own_bus.to_excel("OWNERS_BUSINESS_ONLY.xlsx",header=True, index = False)

everyone_bus.to_csv("OWNERS_BUSINESS_EVERY1.csv",header=True, index = False)
everyone_bus.to_excel("OWNERS_BUSINESS_EVERY1.xlsx",header=True, index = False)

In [158]:
own_bus

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION,Owner First Name,Owner Middle Initial,Owner Last Name,Title,Name
0,2431696-20170116,2500207,377912,2,LIMRA WIRELESS INC.,WIRELESS WORLD,2415 W DEVON AVE 1ST,CHICAGO,IL,60659,...,NaN,43,41.99760701,-87.69053302,"(41.99760701293176, -87.69053301605958)",MAIMLINA,,HASSAN,PRESIDENT,MAIMLINA HASSAN
1,2206583-20170116,2499322,281021,1,CHICAGO WATER CONNECTION,CHICAGO WATER CONNECTION,1914 N LECLAIRE AVE,CHICAGO,IL,60639,...,NaN,NaN,41.91548724,-87.75301232,"(41.915487240332126, -87.75301232003028)",TOMASZ,,GUSTAN,PRESIDENT,TOMASZ GUSTAN
2,23382-20170116,2498097,29722,1,IDEAL DENTAL LAB,IDEAL DENTAL LAB,6212 W GRAND AVE 1ST,CHICAGO,IL,60639,...,NaN,NaN,41.92167131,-87.78103836,"(41.921671312306444, -87.7810383618699)",JOHN,D,LARSON,PRESIDENT,JOHN D LARSON
3,1953514-20170116,2498309,60029,3,RAUL ROBLES,A & R SHOES,3278 W ARMITAGE AVE 1,CHICAGO,IL,60647,...,NaN,NaN,41.917471,-87.71021105,"(41.9174709975593, -87.71021105089828)",RAUL,,ROBLES,SOLE PROPRIETOR,RAUL ROBLES
4,2225555-20170216,2505112,372342,1,LINCOLN PARK SWIM SCHOOL LLC,GOLDFISH SWIM SCHOOL CHICAGO,2630 W BRADLEY PL 1ST,CHICAGO,IL,60618,...,NaN,38,41.94956434,-87.69441251,"(41.9495643375966, -87.69441251019204)",ROBERT,N,RYAN,MANAGING MEMBER,ROBERT N RYAN
5,2225555-20170216,2505112,372342,1,LINCOLN PARK SWIM SCHOOL LLC,GOLDFISH SWIM SCHOOL CHICAGO,2630 W BRADLEY PL 1ST,CHICAGO,IL,60618,...,NaN,38,41.94956434,-87.69441251,"(41.9495643375966, -87.69441251019204)",KATHERINE,,RYAN,MANAGING MEMBER,KATHERINE RYAN
6,1767580-20170216,2502905,209980,3,SHAMSHUN YAROU,ARONSON'S FOR BEAUTY,2913 1/2 W DEVON AVE,CHICAGO,IL,60659,...,NaN,43,41.99739733,-87.70263958,"(41.99739733091123, -87.70263957946979)",SHAMSHUN,,YAROU,SOLE PROPRIETOR,SHAMSHUN YAROU
7,2507012-20170117,2507012,412891,1,IPROPERTYMANAGE INC.,IPROPERTYMANAGE,1977 W 111TH ST 2ND 2F,CHICAGO,IL,60643,...,NaN,NaN,41.69183242,-87.67130676,"(41.69183241616139, -87.67130676318894)",CHERYL,,JOHNSON,PRESIDENT,CHERYL JOHNSON
8,2354689-20170216,2502850,86553,10,"SIDE BY SIDE, INC.",SIDETRACK,3345-3355 N HALSTED ST 1ST & 2ND,CHICAGO,IL,60657,...,NaN,8,41.9431326,-87.64921767,"(41.94313259451181, -87.64921766458053)",ARTHUR,L,JOHNSTON,PRESIDENT,ARTHUR L JOHNSTON
9,2065314-20170116,2499869,357907,1,"TT Gov-Assist,Inc. d/b/a MAKEYOURBODY.com",MAKEYOURBODY.COM,2218 W BELMONT AVE 1ST,CHICAGO,IL,60618,...,NaN,NaN,41.93961358,-87.68400064,"(41.93961357512461, -87.68400063486028)",THEODORA,,TSATSOS,PRESIDENT,THEODORA TSATSOS


In [151]:
test = qgrid.show_grid(everyone_bus, show_toolbar= True, grid_options={'forceFitColumns':False, 'maxVisibleRows': 15})
test

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [152]:
everyone_bus.loc[everyone_bus["Name"] == "W.T.  EGGLESTON"]["DOING BUSINESS AS NAME"].value_counts()

ALLIED WASTE OF CHICAGO                       28
NATIONAL WASTE SERVICES                       12
ALLIED WASTE TRANSPORATION, INC                7
ALLIED WASTE TRANSPORTATION                    6
ALLIED WASTE TRANSPORTATION, INC.              6
REPUBLIC SERVICES OF CHICAGO -  TRK # 1150     1
REPUBLIC SERVICES OF CHICAGO - TRK # 1153      1
REPUBLIC SERVICES OF CHICAGO - TRK # 1152      1
REPUBLIC SERVICES OF CHICAGO - TRK # 1151      1
REPUBLIC SERVICES OF CHICAGO - TRK # 1155      1
REPUBLIC SERVICES OF CHICAGO - TRK # 1154      1
ALLIED WASTE TRANSPORTATION INC                1
Name: DOING BUSINESS AS NAME, dtype: int64

In [41]:
bus_df.loc[bus_df["DOING BUSINESS AS NAME"] == "ALLIED WASTE TRANSPORTATION, INC."].

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
25552,2205511-20171216,2561687,66610,171,"ALLIED WASTE TRANSPORTATION, INC.","ALLIED WASTE TRANSPORTATION, INC.",2608 S DAMEN AVE 1 1,CHICAGO,IL,60608,...,12/16/2017,12/15/2019,11/14/2017,11/15/2017,AAI,NaN,NaN,41.84451147,-87.67578804,"(41.844511469065175, -87.67578803912177)"
25576,2205501-20171216,2561679,66610,160,"ALLIED WASTE TRANSPORTATION, INC.","ALLIED WASTE TRANSPORTATION, INC.",2608 S DAMEN AVE 1 1,CHICAGO,IL,60608,...,12/16/2017,12/15/2019,11/14/2017,11/15/2017,AAI,NaN,NaN,41.84451147,-87.67578804,"(41.844511469065175, -87.67578803912177)"
25618,2205503-20171216,2561681,66610,162,"ALLIED WASTE TRANSPORTATION, INC.","ALLIED WASTE TRANSPORTATION, INC.",2608 S DAMEN AVE 1 1,CHICAGO,IL,60608,...,12/16/2017,12/15/2019,11/14/2017,11/15/2017,AAI,NaN,NaN,41.84451147,-87.67578804,"(41.844511469065175, -87.67578803912177)"
25729,2205502-20171216,2561680,66610,161,"ALLIED WASTE TRANSPORTATION, INC.","ALLIED WASTE TRANSPORTATION, INC.",2608 S DAMEN AVE 1 1,CHICAGO,IL,60608,...,12/16/2017,12/15/2019,11/14/2017,11/15/2017,AAI,NaN,NaN,41.84451147,-87.67578804,"(41.844511469065175, -87.67578803912177)"
25765,2205500-20171216,2561678,66610,159,"ALLIED WASTE TRANSPORTATION, INC.","ALLIED WASTE TRANSPORTATION, INC.",2608 S DAMEN AVE 1 1,CHICAGO,IL,60608,...,12/16/2017,12/15/2019,11/14/2017,11/15/2017,AAI,NaN,NaN,41.84451147,-87.67578804,"(41.844511469065175, -87.67578803912177)"
25785,2205499-20171216,2561677,66610,158,"ALLIED WASTE TRANSPORTATION, INC.","ALLIED WASTE TRANSPORTATION, INC.",2608 S DAMEN AVE 1 1,CHICAGO,IL,60608,...,12/16/2017,12/15/2019,11/14/2017,11/15/2017,AAI,NaN,NaN,41.84451147,-87.67578804,"(41.844511469065175, -87.67578803912177)"


In [156]:
own_bus.groupby(["Name","LEGAL NAME"]).count().sort_values(by = "ID",ascending=False)

,,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,WARD,...,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION,Owner First Name,Owner Middle Initial,Owner Last Name,Title
Name,LEGAL NAME,,,,,,,,,,,,,,,,,,,,,
RICHARD ASHWORTH,WALGREEN CO.,218,218,218,218,218,218,218,218,218,218,...,218,0,104,218,218,218,218,218,218,218
JOHN RICCHIUTO,SP PLUS CORPORATION,198,198,198,198,198,198,198,198,198,198,...,198,0,20,196,196,196,198,198,198,198
ROBERT M TOY,SP PLUS CORPORATION,198,198,198,198,198,198,198,198,198,198,...,198,0,20,196,196,196,198,198,198,198
RICHARD ASHWORTH,"BOND DRUG COMPANY OF ILLINOIS, LLC",141,141,141,141,141,141,141,141,141,141,...,141,0,50,138,138,138,141,141,141,141
Kevin Johnson,STARBUCKS CORPORATION,126,126,126,126,126,126,126,126,126,126,...,126,0,30,126,126,126,126,126,126,126
THOMAS S MOFFATT,"HIGHLAND PARK CVS, L.L.C.",119,119,119,119,119,119,119,119,119,119,...,119,0,45,119,119,119,119,119,119,119
LONNIE W MCCAFFETY,"FAMILY DOLLAR, INC.",118,118,118,118,118,118,118,118,118,118,...,118,0,46,118,118,118,118,118,118,118
BRUCE A WALTERS,"FAMILY DOLLAR, INC.",118,118,118,118,118,118,118,118,118,118,...,118,0,46,118,118,118,118,118,118,118
ADRIAN LLEWELYN MEREDITH,"COMPASS GROUP USA, INC.",110,110,110,110,110,110,110,110,109,109,...,110,0,17,109,109,109,110,110,110,110


In [154]:
own_bus.groupby(["DATE ISSUED","DOING BUSINESS AS NAME"]).count().sort_values(by = "ID",ascending=True)

ID  LICENSE ID  \
DATE ISSUED DOING BUSINESS AS NAME                                             
2016-03-16  T5G CONSTRUCTION, INC.                             1           1   
2018-04-04  Busted Bra Shop                                    1           1   
            CHITOWN SANDWICH CLUB                              1           1   
            Center for Law and Social Work                     1           1   
            ConnectED Consulting Services LLC                  1           1   
            DEL SOL GENERAL SERVICES CORP.                     1           1   
            DIANA'S HAIR DESIGNS                               1           1   
            DISCOUNT ROOFING MATERIALS INC                     1           1   
            Bright Arrow                                       1           1   
            DQ LUXURY REFLEXOLOGY MASSAGE RELAXATION RETREAT   1           1   
            EGASA, INC.                                        1           1   
            ENVIRONMENTAL CLEANSING CORP                       1           1   
            EXEC. 47                                           1           1   
            FACET COLORS JEWELRY                               1           1   
            FILE AWAY NAIL SPA LLC                             1           1   
            FLY LIFE                                           1           1   
            FRASER CONSTRUCTION INC                            1           1   
            DYNAMIC SALON, SPA & BOUTIQUE CO.                  1           1   
            BEED JEWELERY INC                                  1           1   
            BAUMAN HEATING AND COOLING                         1           1   
            BANC OF AMERICA LEASING & CAPITAL, LLC             1           1   
2018-04-03  TRINITY ROOFING                                    1           1   
            Truesdale Coaching and Consulting LLC              1           1   
            URBAN INNOVATIONS, LTD.                            1           1   
            WICKLOW DEVELOPMENT COMPANY                        1           1   
            WORLDBRIDGE PARTNERS                               1           1   
            XOCOME ANTOJERIA                                   1           1   
            ZENAILS, LTD.                                      1           1   
2018-04-04  1914 CLUB                                          1           1   
            5 STAR CHILDCARE DEVELOPMENT CENTER INC.           1           1   
...                                                           ..         ...   
2018-04-25  PUBLIC STORAGE                                    28          28   
2018-10-26  BROWN BROTHERS HARRIMAN & CO.                     28          28   
2017-10-02  BEATRIX                                           28          28   
2017-11-15  ALLIED WASTE OF CHICAGO                           28          28   
2017-10-02  SUMMER HOUSE SANTA MONICA/STELLA BARRA PIZZERIA   30          30   
2018-06-29  LAZ PARKING CHICAGO                               30          30   
2017-05-19  THREE DOTS AND A DASH                             30          30   
2018-03-22  HANDCUT FOODS                                     32          32   
2017-12-05  HUDSON NEWS                                       34          34   
2018-04-24  RPM STEAK                                         36          36   
2017-08-07  ABM Parking Services                              36          36   
2018-06-26  INTERPARK                                         36          36   
2018-10-19  BUB CITY                                          36          36   
2018-04-17  DANA HOTEL AND SPA                                36          36   
2017-04-17  Chipotle Mexican Grill                            38          38   
2018-06-01  CorePower Yoga                                    40          40   
2017-09-22  RADISSON BLU AQUA                                 42          42   
2018-01-09  STANDARD PARKING                                  